## Examples 3 4 5 of week 3 

In [1]:
# load the libraries we'll need
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sys
np.set_printoptions(suppress=True)

In [3]:
# load the data
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_input = tf.keras.utils.to_categorical(y_train)
x_input = (np.reshape(x_train, (x_train.shape[0], 784)) / 255.0).astype(np.float32)

# just use ten samples
x_input = x_input[:10]
y_input = y_input[:10]

# create the variables
W = tf.Variable(tf.zeros([784,10]),name='W')
b = tf.Variable(tf.zeros([10]),name='b')

# set-up the functions for the model
# predicted probability for each class
def y_pred(x):
    # add a print statement using the standard python function
    print(b[0])
    return tf.nn.softmax(tf.matmul(x,W) + b)

# cross entropy loss function
@tf.function
def loss(x,y):  
    y_ = y_pred(x)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))


# set up the optimizer
train_steps = 5
lr = 1e-1
optimizer = tf.optimizers.SGD(lr)

# run the training with the print statement
for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(x_input,y_input)
    gradients = tape.gradient(current_loss, [W, b])
    train_step = optimizer.apply_gradients(zip(gradients, [W ,b]))

2023-05-28 20:45:38.288062: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 20:45:38.845959: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10794 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


Tensor("strided_slice:0", shape=(), dtype=float32)


Although we have added the print statement, it is only being printed once when the function is first called.
It is also only displaying the shape and datatype and not its actual value as we would like. This is because we are using graph mode, so we will have to change the code to use the tf.print
function

In [4]:
# predicted probability for each class
def y_pred(x):
    # use the tf.print function to print to stdout
    tf.print(b[0], output_stream=sys.stdout)   #changed here!
    return tf.nn.softmax(tf.matmul(x,W) + b)

# cross entropy loss function
@tf.function
def loss(x,y):
    y_ = y_pred(x)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))


# set up the optimizer
train_steps = 5
lr = 1e-1
optimizer = tf.optimizers.SGD(lr)


# run the training with the print statement
for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(x_input,y_input)
        gradients = tape.gradient(current_loss, [W, b])
        train_step = optimizer.apply_gradients(zip(gradients, [W ,b]))

-0.00149583095
-0.0019557036
-0.0023749636
-0.00274300273
-0.00306231133


## Example 4 of when learning rate is too high

In [5]:
# learning rate
lr = 1e2

# gradient descent optimizer
optimizer = tf.optimizers.SGD(lr)

# include an operation to print b
for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(x_input,y_input)
        gradients = tape.gradient(current_loss, [W, b])
        train_step = optimizer.apply_gradients(zip(gradients, [W ,b]))

-0.00333956606
-0.245597571
nan
nan
nan


## Example 5: add assert statement

In [11]:
# predicted probability for each class
def y_pred(x):
    tf.debugging.assert_all_finite(W, 'stopping - weights values contain nan')
    tf.debugging.assert_all_finite(b, 'stopping - bias values contain nan')
    return tf.nn.softmax(tf.matmul(x,W) + b)

# cross entropy loss function
@tf.function
def loss(x,y):
    y_ = y_pred(x)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))

# learning rate
lr = 1e2

# gradient descent optimizer
optimizer = tf.optimizers.SGD(lr)

#Now, when we run the code it immediately raises an error as soon as a nan occurs,
for i in range(train_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(x_input,y_input)
        gradients = tape.gradient(current_loss, [W, b])
        train_step = optimizer.apply_gradients(zip(gradients, [W ,b]))

2023-05-28 21:08:19.763934: E tensorflow/core/kernels/check_numerics_op.cc:292] abnormal_detected_host @0x2030c0200 = {1, 0} stopping - weights values contain nan


InvalidArgumentError: Graph execution error:

Detected at node 'VerifyFinite/CheckNumerics' defined at (most recent call last):
    File "/opt/conda/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/opt/conda/lib/